In [2]:
import cv2 as cv
import numpy as np
import math
from matplotlib import pyplot as plt
from functools import cmp_to_key
import glob, os

In [3]:
def count_diff(point1, point2):
    dx = point1[0] - point2[0]
    dy = point1[1] - point2[1]
    return dx * dx + dy * dy


def rotate(rectengles: np.ndarray):
    max_x = rectengles[0][0]
    min_x = rectengles[0][0]
    max_y = rectengles[0][1]
    min_y = rectengles[0][1]
    for rectengle in rectengles:
        width = rectengle[2] - rectengle[0]
        height = rectengle[3] - rectengle[1]
        if (rectengle[0] < min_x):
            min_x = rectengle[0]
        if (rectengle[2] > max_x):
            max_x = rectengle[2]
        if (rectengle[1] < min_y):
            min_y = rectengle[1]
        if (rectengle[3] > max_y):
            max_y = rectengle[3]
    



    top_left: np.ndarray = [rectengles[0][0], rectengles[0][0]]
    top_left_diff = count_diff(top_left, [min_x, min_y])
    top_right: np.ndarray = [rectengles[0][0], rectengles[0][0]]
    top_right_diff = count_diff(top_right, [max_x, min_y])
    bot_left: np.ndarray = [rectengles[0][0], rectengles[0][0]]
    bot_left_diff = count_diff(bot_left, [min_x, max_y])
    bot_right: np.ndarray = [rectengles[0][0], rectengles[0][0]]
    bot_right_diff = count_diff(bot_right, [max_x, max_y])
    for rectengle in rectengles:
        width = rectengle[2] - rectengle[0]
        height = rectengle[3] - rectengle[1]
        if (count_diff([min_x, min_y],[rectengle[0], rectengle[1]]) < top_left_diff):
            top_left_diff = count_diff([min_x, min_y],[rectengle[0], rectengle[1]])
            top_left[0] = rectengle[0]
            top_left[1] = rectengle[1]
        if (count_diff([max_x, min_y], [rectengle[2], rectengle[1]]) < top_right_diff):
            top_right_diff = count_diff([max_x, min_y], [rectengle[2], rectengle[1]])
            top_right[0] = rectengle[0] + width
            top_right[1] = rectengle[1]
        if (count_diff([min_x, max_y], [rectengle[0], rectengle[3]]) < bot_left_diff):
            bot_left_diff = count_diff([min_x, max_y], [rectengle[0], rectengle[3]])
            bot_left[0] = rectengle[0]
            bot_left[1] = rectengle[1] + height
        if (count_diff([max_x, max_y], [rectengle[2], rectengle[3]]) < bot_right_diff):
            bot_right_diff = count_diff([max_x, max_y], [rectengle[2], rectengle[3]])
            bot_right[0] = rectengle[2]
            bot_right[1] = rectengle[3]

    dy = bot_left[1] - bot_right[1]
    sin = (-dy) / (count_diff(bot_left,bot_right) ** 0.5)
    cos = (1 - sin ** 2) ** 0.5
    matrix: np.ndarray = [[cos, -sin], 
                          [sin, cos]] 
    
    for i in range(rectengles.shape[0]):
        width = rectengles[i][2] - rectengles[i][0]
        height = rectengles[i][3] - rectengles[i][1]
        rec = rectengles[i][0:2]
        rec[0] = rec[0] - (max_x + min_x) / 2
        rec[1] = (max_y + min_y) / 2 - rec[1]
        rec = np.dot(matrix, rec)
        rec[0] = rec[0] + (max_x + min_x) / 2
        rec[1] = (max_y + min_y) / 2 - rec[1]
        rectengles[i][0] = rec[0]
        rectengles[i][1] = rec[1]
        rectengles[i][2] = rec[0] + width
        rectengles[i][3] = rec[1] + height
    return sin, max_x, min_x, max_y, min_y

def derotate(sin, max_x, min_x, max_y, min_y, cell_lines):
    sin = -sin
    cos = (1 - sin ** 2) ** 0.5
    matrix: np.ndarray = [[cos, -sin], 
                          [sin, cos]] 
    
    for i in range(len(cell_lines)):
        for j in range(cell_lines[i].shape[0]):
            
            width = cell_lines[i][j][2] - cell_lines[i][j][0]
            height = cell_lines[i][j][3] - cell_lines[i][j][1]
            rec = cell_lines[i][j][0:2]
            rec[0] = rec[0] - (max_x + min_x) / 2
            rec[1] = (max_y + min_y) / 2 - rec[1]
            rec = np.dot(matrix, rec)
            rec[0] = rec[0] + (max_x + min_x) / 2
            rec[1] = (max_y + min_y) / 2 - rec[1]
            cell_lines[i][j][0] = rec[0]
            cell_lines[i][j][1] = rec[1]
            cell_lines[i][j][2] = rec[0] + width
            cell_lines[i][j][3] = rec[1] + height

            
        

In [77]:
def compare(a ,b):
    return a[0] - b[0]
def calc_angle(point1, point2):
    sin = (point2[1] - point1[1]) / (count_diff(point1, point2) ** 0.5)
    return abs(math.asin(sin))


def detect_one_cell_line(points: np.ndarray):
    res = [0]
    while len(res) != len(points):
        cur = 0
        prev = res[-1]
        for i in range(len(points)):
            if (i == prev):
                continue
            diff = count_diff(points[prev], points[i])
            angle = calc_angle(points[prev], points[i])
            if (points[i][0] < points[prev][0]):
                continue
            if (angle < (math.pi / 28)):
                if (diff < count_diff(points[cur], points[prev]) or cur == 0):
                    cur = i
        if (cur == 0):
            break
        else:
            res.append(cur)
    reversed_res = [0]
    while (len(reversed_res) + len(res)) != len(points):
        cur = 0
        prev = reversed_res[-1]
        for i in range(len(points)):
            if (i == prev):
                continue
            diff = count_diff(points[prev], points[i])
            angle = calc_angle(points[prev], points[i])
            if (points[i][0] > points[prev][0]):
                continue
            if (angle < (math.pi / 28)):
                if (diff < count_diff(points[cur], points[prev]) or cur == 0):
                    cur = i
        if (cur == 0):
            break
        else:
            reversed_res.append(cur)

    return reversed_res + res

def rec2point(rectengles: np.ndarray):
    points: np.ndarray = np.ndarray([rectengles.shape[0], 2])
    for i in range(rectengles.shape[0]):
        points[i] = [(rectengles[i][0] + rectengles[i][2]) / 2, (rectengles[i][1] + rectengles[i][3]) / 2]
    return points

def detect_cell_lines(rectengles: np.ndarray) -> np.ndarray:
    
    current_points = rec2point(rectengles)
    current_rectengles = rectengles
    res = []
    while (len(current_points) != 0):
        cell_line = detect_one_cell_line(current_points)
        res.append(current_rectengles[np.array(cell_line)])
        current_rectengles = np.delete(current_rectengles, cell_line, axis = 0)
        current_rectengles = np.array(sorted(current_rectengles, key = cmp_to_key(compare)))
        current_points = rec2point(current_rectengles)
    
    return res





In [5]:
#for file_name in glob.glob("*.bmp"):
#    image = cv.imread(file_name)
#    for row in range(image.shape[0]):
#        for col in range(image.shape[1]):
#            b = image[row, col, 0]
#            g = image[row, col, 1]
#            r = image[row, col, 2]
#            if (g - b > 25 and g - r > 25):
#                image[row, col] = np.array([0, 255, 0])
#            else:
#                image[row, col] = np.array([0, 0, 0])
#    cv.imwrite("./maize_epidermis_results/" + file_name, image)

In [70]:
def compare(a ,b):
    return a[0] - b[0]
def process_image(file_name: str):
    rectengles = []
    file = open(file_name ,"r")
    size = 0
    for line in file.readlines():
        size+=1
        line_split = line.split(' ')
        rectengle = []
        for l in range(len(line_split)):
            rectengle.append(int(line_split[l]))
        rectengles.append(rectengle)
    rectengles = np.asarray(rectengles)
    file.close()
    sin, max_x, min_x, max_y, min_y = rotate(rectengles)
    rectengles = np.array(sorted(rectengles, key = cmp_to_key(compare)))
    cell_lines = detect_cell_lines(rectengles)
    derotate(sin, max_x, min_x, max_y, min_y, cell_lines)
    image = cv.imread(file_name[0: 16 + 26: 1] + ".bmp")
    ##image  = np.zeros(shape=image.shape, dtype=np.uint8)
    for i in range(len(cell_lines)):
        for rec in cell_lines[i]:
            cv.rectangle(image, (rec[0], rec[1]), (rec[2], rec[3]), ( i * ( 255 / (len(cell_lines) - 1)), i * ( 255 / (len(cell_lines) - 1)), i * ( 255 / (len(cell_lines) - 1))), 3)
    image_cm = cv.applyColorMap(image, cv.COLORMAP_JET)  
    cv.imwrite(file_name[0: 16 + 26: 1] + "_out.bmp", image_cm)

In [80]:
for file_name in glob.glob("./maize_epidermis_results/*.txt"):
    process_image(file_name)

In [78]:
def process_image_draw_sep(file_name: str):
    rectengles = []
    file = open(file_name ,"r")
    size = 0
    for line in file.readlines():
        size+=1
        line_split = line.split(' ')
        rectengle = []
        for l in range(len(line_split)):
            rectengle.append(int(line_split[l]))
        rectengles.append(rectengle)
    rectengles = np.asarray(rectengles)
    file.close()
    sin, max_x, min_x, max_y, min_y = rotate(rectengles)
    rectengles = np.array(sorted(rectengles, key = cmp_to_key(compare)))
    cell_lines = detect_cell_lines(rectengles)
    derotate(sin, max_x, min_x, max_y, min_y, cell_lines)
    for i in range(len(cell_lines)):
        image = cv.imread(file_name[0: 16 + 26: 1] + ".bmp")
        for rec in cell_lines[i]:
            cv.rectangle(image, (rec[0], rec[1]), (rec[2], rec[3]), ( 255, 255, 255, 3), 3)
        image_cm = cv.applyColorMap(image, cv.COLORMAP_JET)  
        cv.imwrite(file_name[0: 16 + 26: 1] + "_" + str(i) + "_out.bmp", image_cm)

In [79]:
process_image_draw_sep("./maize_epidermis_results/204_4L_V_1_green_coordinates.txt")